In [7]:
#install kafa package
#pip install kafka-python


In [8]:
#install mongodb package
#!pip install pymongo

###### Démarrer le serveur Kafka et créez un topic que vous utiliserez dans votre projet.

Producer (Producteur) - Envoyer des fichiers CSV à Kafka :

In [7]:
from kafka import KafkaProducer
import csv
import json
import mysql.connector

In [8]:
# Configuration du producteur Kafka
bootstrap_servers = 'localhost:9092'
topic = 'news'

producer = KafkaProducer(bootstrap_servers=bootstrap_servers,
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))


In [9]:
# Fonction pour lire un fichier CSV et envoyer chaque ligne à Kafka
# Définition de la fonction pour produire des lignes de fichier CSV vers Kafka
def produce_csv_to_kafka(file_path):
    # Ouvre le fichier CSV en mode lecture, en spécifiant l'encodage utf-8
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        # Crée un objet DictReader pour lire le fichier CSV avec la première ligne comme en-têtes de colonnes
        csv_reader = csv.DictReader(csvfile)
        # Itère sur chaque ligne du fichier CSV
        for row in csv_reader:
            # Envoie la ligne actuelle à Kafka en utilisant le producteur
            producer.send(topic, value=row)
            # Force l'envoi immédiat du message à Kafka
            producer.flush()


In [20]:
# Exemple d'utilisation
file_path="C:\\Users\\user\\Anaconda-Files\\EUR1H.csv"
produce_csv_to_kafka(file_path)

###### Démarer kafkaConsumer

In [32]:
from kafka import KafkaConsumer
import json
from datetime import datetime

In [18]:
# Remplacez ces valeurs par vos informations Kafka et MySQL
KAFKA_BOOTSTRAP_SERVERS = 'localhost:9092'
KAFKA_TOPIC = 'news'
DB_USER = 'root'
DB_PASSWORD = 'root'
DB_NAME = 'entrepot'


In [26]:
# Configuration du consommateur Kafka
consumer = KafkaConsumer(KAFKA_TOPIC,
                         bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
                         value_deserializer=lambda v: json.loads(v.decode('utf-8')),
                         auto_offset_reset='earliest')  # Ajoutez cette option pour récupérer les messages depuis le début

In [27]:
# Configuration de la base de données MySQL
db_conn = mysql.connector.connect(host='localhost', user=DB_USER, password=DB_PASSWORD, database=DB_NAME)
db_cursor = db_conn.cursor()


In [33]:

try:
    # Consommation des cinq premiers messages Kafka et insertion dans MySQL
    count = 0
    for message in consumer:
        data = message.value

   # Convertir la chaîne de date en format MySQL
        gmt_value = data.get('Gmt time', '')
        if gmt_value:
            gmt_datetime = datetime.strptime(gmt_value, '%d.%m.%Y %H:%M:%S.%f')
            gmt_mysql_format = gmt_datetime.strftime('%Y-%m-%d %H:%M:%S')
            insert_query = f"INSERT INTO news (Gmt, open, high, low, close, volume) VALUES ('{gmt_mysql_format}', {data.get('Open', 0)}, {data.get('High', 0)}, {data.get('Low', 0)}, {data.get('Close', 0)}, {data.get('Volume', 0)})"
            db_cursor.execute(insert_query)
            db_conn.commit()

        count += 1
        if count >= 5:
            break  # Sortir de la boucle après avoir consommé les cinq premiers messages

except KeyboardInterrupt:
    print("L'exécution a été interrompue manuellement.")

# Fermeture des connexions
db_cursor.close()
db_conn.close()


###### inserer un ligne avec python

In [6]:
# Configuration de la connexion à MySQL
from datetime import datetime
# Connexion à la base de données MySQL
conn = mysql.connector.connect(host='localhost', user='root', password='root', database='entrepot')
cursor = conn.cursor()

# Données à insérer
date_time = datetime.strptime('08.07.2020 02:00:00.000', '%d.%m.%Y %H:%M:%S.%f')
open_value = 1.12719
high_value = 1.12796
low_value = 1.12709
close_value = 1.12786
volume_value = 3594.1399

# Requête SQL d'insertion
insert_query = f"INSERT INTO news (Gmt,open, high, low, close, volume) VALUES ('{date_time}', {open_value}, {high_value}, {low_value}, {close_value}, {volume_value})"

# Exécution de la requête
cursor.execute(insert_query)

# Commit des changements
conn.commit()

# Fermeture des connexions
cursor.close()
conn.close()

print("La ligne a été insérée avec succès dans la table 'news' en utilisant une requête simple INSERT.")

La ligne a été insérée avec succès dans la table 'news' en utilisant une requête simple INSERT.
